In [1]:
# INIT

import random
import collections
import numpy as np
import plotly
import plotly.plotly as py
import plotly.tools as tls
import matplotlib.pyplot as plt
import pandas as pd
import time
import datetime
import scipy as sp
import copy as cp
import pickle
import diffusion_functions as dm
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.preprocessing import normalize
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV, train_test_split
from pygsp import graphs, filters, plotting
from jupyterthemes import jtplot
from scipy.linalg import eig
from IPython import display
import warnings

plotting.BACKEND = 'matplotlib'
# plt.rcParams['figure.figsize'] = (8.0, 4.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['font.family'] = 'Times New Roman'
# plt.rcParams['font.size'] = 12
# plt.rcParams['image.cmap'] = 'gray'

jtplot.style(grid=False, figsize=(15, 10))
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12

np.set_printoptions(precision=3,linewidth=250,suppress=True)


>> Functions loaded from diffusion_functions.py


In [2]:
plt.rcParams['text.usetex'] = True
jtplot.style(grid=False, figsize=(8, 6))
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['font.family'] = 'Times New Roman'

fontsize = 12
plt.rcParams['font.size'] = fontsize
plt.rcParams['legend.fontsize'] = fontsize
plt.rcParams['axes.titlesize'] = fontsize
plt.rcParams['axes.labelsize'] = fontsize
# plt.rcParams['mathtext.fontset'] = 'custom'
# plt.rcParams['mathtext.rm'] = 'DejaVu Sans'
# plt.rcParams['mathtext.it'] = 'asd:italic'
# plt.rcParams['mathtext.bf'] = 'DejaVu Sans:bold'

In [3]:
# GSP

# Adjacency matrix functions
#Extended adjacency matrix based on diffusion distances
def extended(M,t,rho=None):
    N = M.shape[1]
    Mt = np.linalg.matrix_power(M,t)
    dds_pre = np.sum((np.kron(Mt,np.ones((N,1))) - np.kron(np.ones((N,1)),Mt))**2,axis=1)
    dds = N*np.reshape(dds_pre,(N,N));
    if not rho:
        rho = 2*np.var(dds_pre)
    At = M + np.exp(-dds/(rho*N))
    np.fill_diagonal(At,0)
    return At

#RBF-kernel adjacency matrix from positions
def kernel_dist(A,pos,rho=None):
    N = A.shape[1]
    W = np.zeros((N,N))
    if not rho:
        rho = 2*np.mean(np.var(pos,axis=1))
    for i in range(N):
        for j in range(N):
            W[i,j] = np.exp(-np.linalg.norm(pos[i,:]-pos[j,:])**2/rho)*A[i,j]
            W[j,i] = W[i,j]
    return W

#Dijkstra algorithm computes shortest paths
def dijkstra(W, source):
    N = len(W)
    vertices = range(N);
    with np.errstate(divide='ignore'):
        D = 1/W
    #vertices, edges = graph
    dist = dict()
    previous = dict()
    hops = dict()

    for vertex in vertices:
        dist[vertex] = float("inf")
        previous[vertex] = -1
        hops[vertex] = 0

    dist[source] = 0
    Q = list(vertices)

    while len(Q) > 0:
        #u = minimum_distance(dist, Q)
        dists = [dist[l] for l in Q]
        u = Q.pop(np.argmin(dists))
#         print('Currently considering', u, 'with a distance of', dist[u])

        if dist[u] == float('inf'):
            break

        #n = get_neighbours(graph, u)
        n = list(np.nonzero(W[u,:])[0])
        for vertex in n:
            alt = dist[u] + D[u, vertex]
            if alt < dist[vertex]:
                dist[vertex] = alt
                previous[vertex] = u
                
    for v in range(N):
        u = v*1
        while previous[u]!=-1:
            u = previous[u]
            hops[v]=hops[v]+1

    return previous,hops,dist
        
# Extended adjacency matrix based on shortest-paths    
def Spath(W,hops):
    dijk = dict();
    for n in range(N):
        dijk[n] = {'prev':{},'hops':{},'dist':{}}
        dijk[n]['prev'],dijk[n]['hops'],dijk[n]['dist'] = dijkstra(W,n)
    Spath = np.zeros((N,N))
    for hop in range(1,hops+1):
        for n in range(N):
            Spath[n,:] = Spath[n,:] + (np.array(dijk[n]['hops'].values())==hop)*np.array(dijk[n]['dist'].values())
    
    return np.divide(1.,Spath,out=np.zeros_like(Spath), where=Spath!=0)

# Computes Laplacian from adjacency
def getL(A):
    return np.diag(A.sum(axis=0)) - A


#COMPUTES GFT
def GFT(Shift,signal):
    #Decomposition
    M_values, M_vr = np.linalg.eigh(Shift)

    #Sorting (ASCENDING)
    idx = np.argsort(M_values)
    M_values = M_values[idx]
    M_vr = M_vr[:,idx]

    signal_hat = M_vr.T.dot(signal)
    return signal_hat,M_values
    

In [4]:
#-------------------CLASSIFIERS---------------------#
class Diffusion_GFT_Classifier(BaseEstimator, ClassifierMixin):
    def __init__(self, M, t, lambda_factor=0.3, tau_factor=0.5, rho=0.5):
        self.lambda_factor = lambda_factor
        self.tau_factor = tau_factor
        self.rho = rho
        self.M = M
        self.t = t
        self.K = tau_factor+10

    def fit(self, signalT, anomaly):
        self.Shift_ = extended(M=self.M, t=self.t, rho=self.rho)
        self.L = np.diag(self.Shift_.sum(axis=0))-self.Shift_
        self.lambdas_ = np.linalg.eigvalsh(self.L)
        signal = signalT.transpose()
        healthy = signal[:,anomaly==0]
        coeffs,lambdas = GFT( self.L, healthy)
        mask = lambdas>self.lambda_factor*lambdas.max()
        filtered = np.abs(coeffs[mask,:])
        tau_partials = np.max(filtered,axis=0)
        self.mintau_ = np.min(tau_partials)
        self.maxtau_ = np.max(tau_partials)
        self.tau_ = np.mean(tau_partials) + self.tau_factor*np.std(tau_partials)
        return self
        
    def predict(self, signalT):
        signal = signalT.transpose()
        coeffs, lambdas = GFT(self.L, signal)
        mask = lambdas>self.lambda_factor*lambdas.max()
        filtered = np.abs(coeffs[mask,:])
        tau_predicted = np.max(filtered,axis=0)
        detection = tau_predicted>self.tau_
        return detection
        
    def score(self, signal, anomaly):
        return f1_score(anomaly, self.predict(signal))
    
    def roc_generator(self, signalT, y):
        signal = signalT.transpose()
        coeffs, lambdas = GFT(self.L, signal)
        mask = lambdas>self.lambda_factor*lambdas.max()
        filtered = np.abs(coeffs[mask,:])
        tau_predicted = np.max(filtered,axis=0)
        
        tpr = []
        fpr = []
        for roctau in np.linspace(start=self.mintau_,stop=5*self.maxtau_,num=500):

            detection = tau_predicted>roctau
            cm = confusion_matrix(y, detection)
            cm_norm = cm.astype('float')/np.sum(cm,axis=1)
            tpr.append(cm_norm[1,1])
            fpr.append(cm_norm[0,1])
            
        return tpr,fpr    

#-----------------------------------------------------------#
class Classic_GFT_Classifier(BaseEstimator, ClassifierMixin):
    def __init__(self, L, lambda_factor=0.3, tau_factor=0.5):
        self.lambda_factor = lambda_factor
        self.tau_factor = tau_factor
        self.L = L
        self.lambdas_ = np.linalg.eigvalsh(L)

    def fit(self, signalT, anomaly):
        signal = signalT.transpose()
        healthy = signal[:,anomaly==0]
        coeffs,lambdas = GFT( self.L, healthy)
        
        mask = lambdas>self.lambda_factor*lambdas.max()
        filtered = np.abs(coeffs[mask,:])
        tau_partials = np.max(filtered,axis=0)
        self.mintau_ = np.min(tau_partials)
        self.maxtau_ = np.max(tau_partials)
        self.tau_ = np.mean(tau_partials) + self.tau_factor*np.std(tau_partials)
        return self
        
    def predict(self, signalT):
        signal = signalT.transpose()
        coeffs, lambdas = GFT(self.L, signal)
        mask = lambdas>self.lambda_factor*lambdas.max()
        filtered = np.abs(coeffs[mask,:])
        tau_predicted = np.max(filtered,axis=0)
        detection = tau_predicted>self.tau_
        return detection
        
    def score(self, signal, anomaly):
        return f1_score(anomaly, self.predict(signal))
    
    def roc_generator(self, signalT, y):
        signal = signalT.transpose()
        coeffs, lambdas = GFT(self.L, signal)
        mask = lambdas>self.lambda_factor*lambdas.max()
        filtered = np.abs(coeffs[mask,:])
        tau_predicted = np.max(filtered,axis=0)
        
        tpr = []
        fpr = []
        for roctau in np.linspace(start=self.mintau_,stop=5*self.maxtau_,num=500):

            detection = tau_predicted>roctau
            cm = confusion_matrix(y, detection)
            cm_norm = cm.astype('float')/np.sum(cm,axis=1)
            tpr.append(cm_norm[1,1])
            fpr.append(cm_norm[0,1])
            
        return tpr,fpr           

#-----------------------------------------------------------#
class Markov_GFT_Classifier(BaseEstimator, ClassifierMixin):
    def __init__(self, Transform, lambdas, lambda_factor=0.3, tau_factor=0.5):
        self.lambda_factor = lambda_factor
        self.tau_factor = tau_factor
        self.Transform = Transform
        self.lambdas = lambdas

    def fit(self, signalT, anomaly):
        signal = signalT.transpose()
        healthy = signal[:,anomaly==0]
        coeffs = self.Transform.dot(healthy)        
        mask = self.lambdas<self.lambda_factor*self.lambdas.max()
        filtered = np.abs(coeffs[mask,:])
        tau_partials = np.max(filtered,axis=0)
        self.mintau_ = np.min(tau_partials)
        self.maxtau_ = np.max(tau_partials)
        self.tau_ = np.mean(tau_partials) + self.tau_factor*np.std(tau_partials)
        return self
        
    def predict(self, signalT):
        signal = signalT.transpose()
        coeffs = self.Transform.dot(signal)        
        mask = self.lambdas<self.lambda_factor*self.lambdas.max()
        
        filtered = np.abs(coeffs[mask,:])
        tau_predicted = np.max(filtered,axis=0)
        detection = tau_predicted>self.tau_
        return detection
        
    def score(self, signal, anomaly):
        return f1_score(anomaly, self.predict(signal))         


In [5]:
np.random.seed(seed=0)
N = 100
k_neighbors = 4
disconnected = True
while disconnected:
    features = np.random.rand(N,2)
    s = np.ones((N,1))

    G = graphs.NNGraph(features,k=k_neighbors,center=False, rescale=False)
    disconnected = not G.is_connected()
A = G.A.toarray()*1.
W = G.W.toarray()*1.
L_a = np.diag(A.sum(axis=0)) - A
L_w = np.diag(W.sum(axis=0)) - W

epsilon = 1./(np.max(A.sum(axis=0))*1.25)
M_a = np.eye(N) - epsilon*L_a

n_hops=3
A_sp = np.zeros((N,N,n_hops))
L_sp = np.zeros((N,N,n_hops))
for h in range(1,n_hops+1):
    A_sp[:,:,h-1] = Spath(A,h)
    L_sp[:,:,h-1] = np.diag(A_sp[:,:,h-1].sum(axis=0)) - A_sp[:,:,h-1]
    
#Heimowitz
D = np.diag(A.sum(axis=0))
M = np.linalg.inv(D).dot(A)
lambdas_heimowitz,eigenvectors_heimowitz = np.linalg.eig(M)
sort_idx = np.argsort(lambdas_heimowitz)
lambdas_heimowitz = lambdas_heimowitz[sort_idx]-lambdas_heimowitz.min()
eigenvectors_heimowitz = eigenvectors_heimowitz[:,sort_idx]
transform_heimowitz = np.linalg.inv(eigenvectors_heimowitz)

## MC Runs

In [6]:
warnings.filterwarnings('ignore') 
np.random.seed(seed=1)

### DIFFUSION CLASSIFIER
df1gft_params = {'lambda_factor':[0.1,0.2,0.3,0.4,0.5],
              'tau_factor':np.arange(0.2,2.0,0.1),
              'rho':np.linspace(0.001,0.2,6)}
df1gftclassifier = GridSearchCV(Diffusion_GFT_Classifier(M=M_a,t=1),
                                param_grid=df1gft_params, cv=5, verbose=False)

df2gft_params = {'lambda_factor':[0.2,0.3,0.4,0.5,0.6],
              'tau_factor':np.arange(0.2,2.0,0.1),
              'rho':np.linspace(0.001,0.2,6)}
df2gftclassifier = GridSearchCV(Diffusion_GFT_Classifier(M=M_a,t=2),
                                param_grid=df2gft_params, cv=5, verbose=False)

### CONVENTIONAL GFT CLASSIFIER
gft_params = {'lambda_factor':np.arange(0.1,0.7,0.05),
              'tau_factor':np.arange(0.2,2.1,0.1)}
gftclassifier = GridSearchCV(Classic_GFT_Classifier(L=L_a), gft_params, cv=5, verbose=False)

### SHORTEST PATH CLASSIFIER
sp1classifier = GridSearchCV(Classic_GFT_Classifier(L=L_sp[:,:,0]), gft_params, cv=5, verbose = False)
sp2classifier = GridSearchCV(Classic_GFT_Classifier(L=L_sp[:,:,1]), gft_params, cv=5, verbose = False)
sp3classifier = GridSearchCV(Classic_GFT_Classifier(L=L_sp[:,:,2]), gft_params, cv=5, verbose = False)

### Heimowitz Markov Classifier
mrk_params = {'lambda_factor':np.arange(0.4,0.9,0.05),
              'tau_factor':np.arange(0.2,2.1,0.1)}
mrkclassifier = GridSearchCV(Markov_GFT_Classifier(Transform=transform_heimowitz,lambdas=lambdas_heimowitz), mrk_params, cv=5, verbose=False)


im_size = 200;
xdim = np.linspace(0,1,im_size)
ydim = np.linspace(0,1,im_size)
X,Y = np.meshgrid(xdim,ydim)

noise_level = 0.1
fx = 1
fy = 2
fnx = 5
fny = 6
T_max = 600

number_of_runs = 25
score_df1 = np.zeros(number_of_runs)
score_df2 = np.zeros(number_of_runs)
score_gft = np.zeros(number_of_runs)
score_sp1 = np.zeros(number_of_runs)
score_sp2 = np.zeros(number_of_runs)
score_sp3 = np.zeros(number_of_runs)
score_mrk = np.zeros(number_of_runs)

for run in range(number_of_runs):
    print run
    px = 0
    py = 0
    Xdata = np.zeros((N,T_max))
    Ydata = np.zeros(T_max)

    for t in range(T_max/2):
    #     fx = fx + 0.1*(np.random.rand()-0.5)
        px = px + 0.1*(np.random.rand()-0.5)

    #     fy = fy + 0.05*(np.random.rand()-0.5)
        py = py + 0.05*(np.random.rand()-0.5)

        Xdata[:,t] = np.cos(2*np.pi*fx*features[:,0] + px) + np.cos(2*np.pi*fy*features[:,1] + py)
        Ydata[t] = 0

    for t in range(T_max/2,T_max):
#         fx = fx + 0.1*(np.random.rand()-0.5)
        px = px + 0.1*(np.random.rand()-0.5)

#         fy = fy + 0.05*(np.random.rand()-0.5)
        py = py + 0.05*(np.random.rand()-0.5)

        Xdata[:,t] = np.cos(2*np.pi*fx*features[:,0] + px) + np.cos(2*np.pi*fy*features[:,1] + py) + noise_level*(np.cos(2*np.pi*fnx*features[:,0]) + np.cos(2*np.pi*fny*features[:,1]))
        Ydata[t] = 1

    Xtrain,Xtest,Ytrain,Ytest = train_test_split(Xdata.T,Ydata,test_size=0.5)
#     Xtrain = Xtrain.T
#     Xtest = Xtest.T
    
    df1class = df1gftclassifier.fit(Xtrain,Ytrain)
    df2class = df2gftclassifier.fit(Xtrain,Ytrain)
    gftclass = gftclassifier.fit(Xtrain,Ytrain)
#     sp1class = sp1classifier.fit(Xtrain,Ytrain)
    sp2class = sp2classifier.fit(Xtrain,Ytrain)
    sp3class = sp3classifier.fit(Xtrain,Ytrain)
    mrkclass = mrkclassifier.fit(Xtrain,Ytrain)

#     print df1gftclassifier.best_params_
    score_df1[run] = df1gftclassifier.best_estimator_.score(Xtest,Ytest)
    score_df2[run] = df2gftclassifier.best_estimator_.score(Xtest,Ytest)
    score_gft[run] = gftclassifier.best_estimator_.score(Xtest,Ytest)
#     score_sp1[run] = sp1classifier.best_estimator_.score(Xtest,Ytest)
    score_sp2[run] = sp2classifier.best_estimator_.score(Xtest,Ytest)
    score_sp3[run] = sp3classifier.best_estimator_.score(Xtest,Ytest)
    score_mrk[run] = mrkclassifier.best_estimator_.score(Xtest,Ytest)
    
    print score_gft
    print score_df1
    print score_df2
    print score_sp2
    print score_sp3
    print score_mrk
    
# plt.figure(figsize=(12,8))
# plt.subplot(1,3,1)
# plt.imshow(np.cos(2*np.pi*fx*X + px) + np.cos(2*np.pi*fy*Y + py))
# plt.subplot(1,3,2)
# plt.imshow(0.1*(np.cos(2*np.pi*fnx*X + px) + np.cos(2*np.pi*fny*Y + py)))
# plt.subplot(1,3,3)
# plt.imshow(np.cos(2*np.pi*fx*X + px) + np.cos(2*np.pi*fy*Y + py) + 0.1*(np.cos(2*np.pi*fnx*X + px) + np.cos(2*np.pi*fny*Y + py)))
# plt.show()

0
[ 0.924  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.989  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.997  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.916  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.798  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
1
[ 0.924  

[ 0.924  0.979  0.83   1.     0.941  0.97   0.879  0.872  0.943  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.989  0.989  0.968  1.     0.965  0.993  1.     0.966  0.997  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 1.     0.989  0.993  1.     1.     1.     1.     1.     0.972  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.997  0.887  0.963  0.987  0.875  1.     0.753  0.911  0.987  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.916  0.853  0.99   0.866  0.87   0.964  0.73   0.699  0.552  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.798  0.691  0.257  0.997  0.242  0.664  0.677  0.879  0.908  0.     0.     0.     0.     0.     0.     0.     0

[ 0.924  0.979  0.83   1.     0.941  0.97   0.879  0.872  0.943  0.871  0.876  0.994  0.794  0.915  0.785  0.711  0.939  0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.989  0.989  0.968  1.     0.965  0.993  1.     0.966  0.997  0.95   0.876  0.994  0.989  0.937  0.838  0.966  1.     0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 1.     0.989  0.993  1.     1.     1.     1.     1.     0.972  1.     0.974  1.     0.956  1.     1.     1.     0.986  0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.997  0.887  0.963  0.987  0.875  1.     0.753  0.911  0.987  1.     0.983  0.994  0.923  0.901  0.899  0.997  0.943  0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.916  0.853  0.99   0.866  0.87   0.964  0.73   0.699  0.552  0.932  0.628  0.991  0.902  0.963  0.831  0.677  0.594  0.     0.     0.     0.     0.     0.     0.     0.   ]
[ 0.798  0.691  0.257  0.997  0.242  0.664  0.677  0.879  0.908  0.01   0.568  0.059  0.756  0.587  0.813  0.879  0

[ 0.924  0.979  0.83   1.     0.941  0.97   0.879  0.872  0.943  0.871  0.876  0.994  0.794  0.915  0.785  0.711  0.939  0.923  1.     0.717  0.963  0.925  0.8    1.     0.92 ]
[ 0.989  0.989  0.968  1.     0.965  0.993  1.     0.966  0.997  0.95   0.876  0.994  0.989  0.937  0.838  0.966  1.     0.99   0.989  0.924  0.963  0.954  0.982  1.     0.942]
[ 1.     0.989  0.993  1.     1.     1.     1.     1.     0.972  1.     0.974  1.     0.956  1.     1.     1.     0.986  1.     0.953  0.809  0.987  0.936  0.983  1.     1.   ]
[ 0.997  0.887  0.963  0.987  0.875  1.     0.753  0.911  0.987  1.     0.983  0.994  0.923  0.901  0.899  0.997  0.943  0.986  0.989  0.658  0.993  1.     1.     0.827  1.   ]
[ 0.916  0.853  0.99   0.866  0.87   0.964  0.73   0.699  0.552  0.932  0.628  0.991  0.902  0.963  0.831  0.677  0.594  0.98   0.846  0.505  0.797  0.687  0.842  0.96   0.912]
[ 0.798  0.691  0.257  0.997  0.242  0.664  0.677  0.879  0.908  0.01   0.568  0.059  0.756  0.587  0.813  0.879  0

In [ ]:
    print score_gft
    print score_df1
    print score_df2
    print score_sp2
    print score_sp3
    print score_mrk

In [ ]:
[0.869 0.996 1.    0.993]
[0.987 0.996 1.    0.993]
[1.    1.    1.    0.993]
[1.    1.    0.856 0.939]
[0.769 0.377 0.86  0.983]
[0.777 0.966 0.879 0.939]

In [ ]:
[1.    0.959 1.    1.   ]
[1.    0.959 1.    1.   ]
[1.    0.996 1.    0.973]
[0.931 1.    0.865 0.877]
[0.229 0.12  0.678 0.192]
[0.953 0.983 0.99  1.   ]

In [ ]:
score_df1.mean()

In [ ]:
np.linspace(0.001,0.2,6)

In [ ]:
print score_df1,'\n\n',score_gft

In [ ]:
print score_sp1,'\n',score_sp2,'\n',score_sp3

In [ ]:
print score_mrk

In [ ]:
print score_df1.mean()
print score_df2.mean()
print score_gft.mean()
print score_sp2.mean()
print score_sp3.mean()
print score_mrk.mean()

In [7]:
scores = {'df1':score_df1, 'gft':score_gft, 'df2':score_df2, 'sp2':score_sp2, 'sp3':score_sp3, 'mrk':score_mrk}

In [8]:
scores

{'df1': array([ 0.989,  0.989,  0.968,  1.   ,  0.965,  0.993,  1.   ,  0.966,  0.997,  0.95 ,  0.876,  0.994,  0.989,  0.937,  0.838,  0.966,  1.   ,  0.99 ,  0.989,  0.924,  0.963,  0.954,  0.982,  1.   ,  0.942]),
 'df2': array([ 1.   ,  0.989,  0.993,  1.   ,  1.   ,  1.   ,  1.   ,  1.   ,  0.972,  1.   ,  0.974,  1.   ,  0.956,  1.   ,  1.   ,  1.   ,  0.986,  1.   ,  0.953,  0.809,  0.987,  0.936,  0.983,  1.   ,  1.   ]),
 'gft': array([ 0.924,  0.979,  0.83 ,  1.   ,  0.941,  0.97 ,  0.879,  0.872,  0.943,  0.871,  0.876,  0.994,  0.794,  0.915,  0.785,  0.711,  0.939,  0.923,  1.   ,  0.717,  0.963,  0.925,  0.8  ,  1.   ,  0.92 ]),
 'mrk': array([ 0.798,  0.691,  0.257,  0.997,  0.242,  0.664,  0.677,  0.879,  0.908,  0.01 ,  0.568,  0.059,  0.756,  0.587,  0.813,  0.879,  0.868,  0.417,  0.009,  0.635,  0.93 ,  0.068,  0.627,  0.29 ,  0.41 ]),
 'sp2': array([ 0.997,  0.887,  0.963,  0.987,  0.875,  1.   ,  0.753,  0.911,  0.987,  1.   ,  0.983,  0.994,  0.923,  0.901,  0.89

In [9]:
outfile = open('scores_2Dsurface_20201604.pkl','wb')
pickle.dump(scores, outfile)
outfile.close()

In [ ]:
score_df1.mean()

In [ ]:
scores_means = np.array((score_gft.mean(),score_df1.mean(),score_df2.mean(),score_sp2.mean(),score_sp3.mean(),score_mrk.mean()))
scores_std = np.array((score_gft.std(),score_df1.std(),score_df2.std(),score_sp2.std(),score_sp3.std(),score_mrk.std()))

In [ ]:
scores_means

In [ ]:
graph_dict = {'features':features}

In [ ]:
graph_dict

In [ ]:
outfile = open('graph_20201504.pkl','wb')
pickle.dump(graph_dict, outfile)
outfile.close()

In [ ]:
plt.plot(scores_means)
plt.show()

In [ ]:
scores_means

In [ ]:
stop here

## Training the classifier

### Method: DDs

In [ ]:
active_scales = [1]

In [ ]:
## TRAINING
df1gft_params = {'lambda_factor':[0.1,0.3,0.5,0.6],
              'tau_factor':np.arange(0.2,2.5,0.1),
              'rho':np.linspace(0.001,0.2,5)}
df1gftclassifier = GridSearchCV(Diffusion_GFT_Classifier(M=M_a,t=1),
                                param_grid=df1gft_params, cv=5, verbose=False, n_jobs=-1)

#################################################################################################

df2gft_params = {'lambda_factor':[0.6,0.7,0.8,0.9],
              'tau_factor':np.arange(0.5,2.5,0.1),
              'rho':np.arange(0.2,0.85,0.1)}
df2gftclassifier = GridSearchCV(Diffusion_GFT_Classifier(M=M_a,t=2),
                                param_grid=df2gft_params, cv=5, verbose=False, n_jobs=-1)

#################################################################################################

df3gft_params = {'lambda_factor':[0.6,0.85,0.9,0.95],
              'tau_factor':np.arange(0.5,2.5,0.1),
              'rho':np.arange(0.2,0.85,0.1)}
df3gftclassifier = GridSearchCV(Diffusion_GFT_Classifier(M=M_a,t=3),
                                param_grid=df3gft_params, cv=5, verbose=False, n_jobs=-1)

In [ ]:
## TRAINING
if np.isin(1,active_scales):
    df1class = df1gftclassifier.fit(Xtrain.transpose(),Ytrain)
    print 'Scale=1'
    print 'Best paramters:', df1gftclassifier.best_params_
    print 'Best f1 score', df1gftclassifier.best_score_
    print ''
    
if np.isin(2,active_scales):
    df2class = df2gftclassifier.fit(Xtrain.transpose(),Ytrain)
    print 'Scale=2'
    print 'Best paramters:', df2gftclassifier.best_params_
    print 'Best f1 score', df2gftclassifier.best_score_
    print ''
if np.isin(3,active_scales):
    df3class = df3gftclassifier.fit(Xtrain.transpose(),Ytrain)
    print 'Scale=3'
    print 'Best paramters:', df3gftclassifier.best_params_
    print 'Best f1 score', df3gftclassifier.best_score_

In [ ]:
## TRAINING (plot confusion matrice)
if np.isin(1,active_scales):
    cm_train1 = confusion_matrix(Ytrain,df1gftclassifier.best_estimator_.predict(Xtrain.transpose()))
    dm.plot_confusion_matrix(cm=cm_train1,classes=['Healthy','Anomaly'],normalize=True)
    plt.show()

if np.isin(2,active_scales):
    cm_train2 = confusion_matrix(Ytrain,df2gftclassifier.best_estimator_.predict(Xtrain.transpose()))
    dm.plot_confusion_matrix(cm=cm_train2,classes=['Healthy','Anomaly'],normalize=True)
    plt.show()

if np.isin(3,active_scales):
    cm_train3 = confusion_matrix(Ytrain,df3gftclassifier.best_estimator_.predict(Xtrain.transpose()))
    dm.plot_confusion_matrix(cm=cm_train3,classes=['Healthy','Anomaly'],normalize=True)
    plt.show()

### Method: Conventional

In [ ]:
gft_params = {'lambda_factor':np.arange(0.1,0.5,0.05),
              'tau_factor':np.arange(0.2,2.1,0.1)}

gftclassifier = GridSearchCV(Classic_GFT_Classifier(L=L_a), gft_params, cv=5, verbose=False)

In [ ]:
gftclass = gftclassifier.fit(Xtrain.transpose(),Ytrain)
print 'Best paramters:', gftclassifier.best_params_
print 'Best f1 score', gftclassifier.best_score_

In [ ]:
cm_train = confusion_matrix(Ytrain,gftclassifier.best_estimator_.predict(Xtrain.transpose()))
cm_train_norm = cm_train.astype('float')/np.sum(cm_train,axis=1)
dm.plot_confusion_matrix(cm=cm_train,classes=['Healthy','Anomaly'],normalize=True)
plt.show()

### Teste

In [ ]:

number_of_runs = 100
score_df1 = np.zeros(number_of_runs)
score_gft = np.zeros(number_of_runs)

for run in range(number_of_runs):
    px = 0
    py = 0
    T_max = 1000
    Xtest = np.zeros((N,T_max))
    Ytest = np.zeros(T_max)

    for t in range(T_max/2):
    #     fx = fx + 0.1*(np.random.rand()-0.5)
        px = px + 0.1*(np.random.rand()-0.5)

    #     fy = fy + 0.05*(np.random.rand()-0.5)
        py = py + 0.05*(np.random.rand()-0.5)

        Xtest[:,t] = np.cos(2*np.pi*fx*features[:,0] + px) + np.cos(2*np.pi*fy*features[:,1] + py)
        Ytest[t] = 0

    for t in range(T_max/2,T_max):
        fx = fx + 0.1*(np.random.rand()-0.5)
        px = px + 0.1*(np.random.rand()-0.5)

        fy = fy + 0.05*(np.random.rand()-0.5)
        py = fy + 0.05*(np.random.rand()-0.5)

        Xtest[:,t] = np.cos(2*np.pi*fx*features[:,0] + px) + np.cos(2*np.pi*fy*features[:,1] + py) + noise_level*(np.cos(2*np.pi*fnx*features[:,0]) + np.cos(2*np.pi*fny*features[:,1]))
        Ytest[t] = 1
        
    score_gft[run] = gftclassifier.best_estimator_.score(Xtest.transpose(),Ytest)
    score_df1[run] = df1gftclassifier.best_estimator_.score(Xtest.transpose(),Ytest)

print np.mean(score_df1),' ',np.std(score_df1)
print np.mean(score_gft),' ',np.std(score_gft)

In [ ]:
score_df1

In [ ]:
score_gft

In [ ]:
1>np.nan

In [ ]:
print 'Step 1 - Test f1 score:', df1gftclassifier.best_estimator_.score(Xtest.transpose(),Ytest)
# print 'Step 2 - Test f1 score:', df2gftclassifier.best_estimator_.score(Xtest.transpose(),Ytest)
# print 'Step 3 - Test f1 score:', df3gftclassifier.best_estimator_.score(Xtest.transpose(),Ytest)
print 'Laplace- Test f1 score:', gftclassifier.best_estimator_.score(Xtest.transpose(),Ytest)

In [ ]:
gftclassifier.best_estimator_.tau_